In [ ]:
#default_exp operators

In [28]:
#export
class Operator():
    def __init__(self,input_var_names:list=[],namePrefix:str = None, observes:bool = False):
        self.__name__ = f"{namePrefix}_{'_'.join(input_var_names)}"
        if type(input_var_names) == str : input_var_names = [input_var_names]
        self._args = [*input_var_names] 
        if len(input_var_names) == 1: self._args = input_var_names[0].split()
        self._observes = observes
        self._wasNeverRun = True
    def firstrun(self,*args):
        pass
    def run(self, *args):
        raise NotImplemented("Operator is abstract, you must subclass it")
    def __call__(self,*args):
        if self._wasNeverRun :
            self._wasNeverRun = False
            self.firstrun(*args)
        return self.run(*args)

class Avg(Operator):
    """
    Compute the running average of a variable of name passed as
    `input_var_name` and output a var named `name` if passed else "Avg_"+`input_var_name`
    
    Args:
    ========================
        * `input_var_name`: str or list<str>
        if str, content is splitted by spaces
        * `name` (optional) : str
    
    """
    def __init__(self,input_var_name):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Avg", observes = True)
        self._running_avg:float = 0
        self._running_count:int = 0
    def run(self,varin):
        self._running_count += 1
        self._running_avg = (varin/self._running_count) + (self._running_avg*((self._running_count-1)/self._running_count) )
        return self._running_avg

class Acc(Operator):
    def __init__(self,input_var_name):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Acc",  observes = True)
        self._running_sum:float = 0
    def run(self,varin):
        self._running_sum += varin
        return self._running_sum

class Exp_avg(Operator):
    def __init__(self,input_var_name, alpha = 0.9):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Exp_avg", observes = True)
        self._running_avg:float = 0
        assert(0<alpha<1)
        self._alpha : float = alpha
        self._1_minus_alpha : float = (1. - alpha)
    def run(self,varin):
        self._running_avg = (self.alpha * self._running_avg) + (self._1_minus_alpha * varin)
        return self._running_avg

class Linear(Operator):
    def __init__(self,input_var_name, gain = 1, bias =0):
        super().__init__(input_var_names = [input_var_name], namePrefix="Linear")
        self.gain = gain
        self.bias = bias
    def run(self,varin): return self.gain * varin + self.bias 

class Add(Operator):
    """Adds every vars passed as imputs"""
    def __init__(self,*input_var_names):
        super().__init__(*input_var_names, namePrefix="Add")
    def run(self,*inputs): 
        ret = 0
        for inp in inputs : ret += inp
        return inp
class Delay(Operator):
    def __init__(self,input_var_name, init_val, delay_in_step:int=1):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Delayed",  observes = True)
        assert(type(delay) == int)
        from collections import deque
        self._past_vals : deque = deque([init_val for i in range(delay_in_step)])
    def run(self,varin):
        self._past_vals.append(varin)
        return self._past_vals.popleft()

class Const(Operator):
    def __init__(self, val):
        super().__init__([], observes = True, namePrefix="Const")
        self._val = val
    def run(self):return self._val



In [33]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted GC.ipynb.
Converted operators.ipynb.


In [ ]:

if len(input_var_names) == 1 : input_var_names[0].split()